# Hi

In [59]:
# Load dependencies
# import numpy as np
from ipycanvas import Canvas,  hold_canvas
canvas = Canvas(width=800, height=800)

## The First Maze
Leverage the "recursive backtracker" algorithm. It is a randomized version of the depth-first search algorithm.

### The Recursive Algorithm
1. Given a current cell as a parameter, Mark the current cell as visited.
2. While the current cell has any unvisited neighbor cells:
  1. Choose one of the unvisited neighbors.
  2. Remove the wall between the current cell and the chosen cell.
  3. Invoke the routine recursively for a chosen cell.

### The Iterative Algorithm
To avoid a deep recursive, a stack can be used.

1. Choose the initial cell, mark it as visited and push it to the stack.
2. While the stack is not empty:
  1. Pop a cell from the stack and make it a current cell.
  2. If the current cell has any neighbors which have not been visited:
    1. Push the current cell to the stack.
    2. Choose one of the unvisited neighbors.
    3. Remove the wall between the current cell and the chosen cell.
    4. Mark the chosen cell as visited and push it to the stack.

### The Data Structures
A maze is composed of traversable "cells". For my purposes a cell is defined as:
- Fixed size. The cell cannot grow or shrink.
- Fixed in place. Cells don't move.
- All cells are the same size.
- Composed of 4 sides: north, south, east, west.


In [60]:
# All class and type definitions
from typing import List
from collections import namedtuple

"""
Represents a traversable room in a maze.
"""
class MazeCell:
  north:bool = True
  south:bool = True
  east:bool = True
  west:bool = True
  visited:bool = True
  
  def __init__(self) -> None:
    pass

  # TODO: Probably should separate drawing logic from the cell structure.
  # Don't want Cell to know about IpyCanvas
  def draw() -> None:
    pass

"""
Convenience tuples for working with cell coordinates. 
"""
Point = namedtuple('Point', ['x','y'])
Corner = Point

class Maze:
  """
  Represents a maze of connected cells. A "cell" is simple a space a person could occupy.
  """
  _grid: List[List[MazeCell]]
  _width: int
  _height: int

  def __init__(self, width: int, height: int) -> None:
    self._grid = []
    self._width = width
    self._height = height
    self._populate()

  @property 
  def width(self) -> int:
    return self._width 

  @property
  def height(self) -> int:
    return self._height

  def _populate(self) -> None:
    """
    Builds a rectangular grid of cells in which all the walls are intially closed.
    """
    for _ in range(self.height):
      row: list[MazeCell] = []
      for _ in range(self.width):
        row.append(MazeCell())
      self._grid.append(row)

  def cell(self, x, y) -> Optional[MazeCell]:
    """
    Finds a cell in the maze by its x,y coordinate.
    The origin of the 2D grid (0,0) is the upper left corner.

    Returns:
      Returns an instance of a MazeCell if it exists, otherwise None.
    """
    found: Optional[MazeCell]
    if (x < 0 or x >= self.width) or (y < 0 or y >= self.height):
      found = None
    else:  
      found = self._grid[y][x]
    return found 


In [61]:
"""
Required Capabilities
- Select an initial cell at random. (done)
- Stack Push/Pop
- Given a cell, query it's neighbors to see if they've been visited
- Remove a wall on the current cell and the adjacent cell.
"""
import random
from collections import deque
from typing import Optional

"""
Given the size of the maze, randomly select a location.
Returns the location in the 2D array of the selected cell.
"""
def pick_random_location(width, height) -> Point:
  return Point(random.randint(0,width-1), random.randint(0,height-1))

class Stack:
  """
  A FILO Queue built with collections.deque (Double Linked List).
  """
  _data: deque
  def __init__(self) -> None:
    self._data = deque()

  def push(self, point: Point) -> None:
    """
    Adds a point to the top of the stack.
    """
    self._data.appendleft(point)

  def pop(self) -> Optional[Point]:
    """
    Returns and removes the last item added to the stack.
    If the stack is empty then the function returns None.
    Check with "if val is not None: ..."
    """
    return self._data.popleft() if len(self._data) > 0 else None


def generate_maze_walls(maze: Maze) -> Maze:
  """
  Traverses the grid of cells creates a maze by opening walls in place.

  Returns:
    The modified grid.
  """
  stack = Stack()
  pick_random_location


In [62]:
# Handle Rendering

def draw_wall(canvas: Canvas, start: Corner, stop: Corner) -> None:
  canvas.move_to(start.x, start.y)
  canvas.line_to(stop.x, stop.y)
  canvas.stroke()

def draw_cell_walls(cell: MazeCell, canvas: Canvas, cell_index: int, row_index: int, room_size_width: int, room_size_height: int) -> None:
  canvas.begin_path()

  upper_left_corner = Corner(cell_index*room_size_width, row_index*room_size_height)
  upper_right_corner = Corner(upper_left_corner.x + room_size_width, upper_left_corner.y)
  lower_right_corner = Corner(upper_right_corner.x, upper_right_corner.y + room_size_height)
  lower_left_corner = Corner(lower_right_corner.x - room_size_width, lower_right_corner.y)

  if cell.north: draw_wall(canvas, upper_left_corner, upper_right_corner)
  if cell.east: draw_wall(canvas, upper_right_corner, lower_right_corner)
  if cell.south: draw_wall(canvas, lower_right_corner, lower_left_corner)
  if cell.west: draw_wall(canvas, lower_left_corner, upper_left_corner)
  
  return

"""Draws the maze."""
def draw_maze(maze: Maze, canvas: Canvas) -> Canvas:
  room_size_width: int =  20
  room_size_height: int = 20
  
  with hold_canvas(canvas):
    canvas.line_width = 5
    for row_index in range(maze.height):
      for cell_index in range(maze.width):
        cell = maze.cell(cell_index, row_index)
        draw_cell_walls(cell, canvas, cell_index, row_index, room_size_width, room_size_height)
  return canvas;

In [63]:
# Generate a maze.
maze: Maze = Maze(40, 40)
generate_maze_walls(maze)
canvas = draw_maze(maze, canvas)
display(canvas)

Canvas(height=800, width=800)